In [1]:
import unittest
from natural20.map import Map, Terrain
from natural20.battle import Battle
from natural20.player_character import PlayerCharacter
from natural20.map_renderer import MapRenderer
from natural20.die_roll import DieRoll
from natural20.generic_controller import GenericController
from natural20.utils.utils import Session
from natural20.actions.move_action import MoveAction
from natural20.action import Action
from natural20.gym.dndenv import dndenv, action_type_to_int
from gymnasium import register, envs, make
from llm_interface import GPT4Interfacer, LLama3Interface
import tqdm
import random

# Get baseline random policy vs baseline AI

In [2]:
MAX_STEPS = 100
NUM_TRAJECTORIES_TO_SAMPLE = 30

In [3]:
env = make("dndenv-v0", root_path="../templates", render_mode="ansi")

In [4]:
total_rewards = []
for trials in range(NUM_TRAJECTORIES_TO_SAMPLE):
    observation, info = env.reset(seed=42)
    terminal = False
    step = 0
    action =  random.choice(info['available_moves'])
    rewards = 0
    while not terminal and step < MAX_STEPS:
        step += 1
        observation, reward, terminal, truncated, info = env.step(action)
        rewards += reward
        if not terminal and not truncated:
            action = random.choice(info['available_moves'])
        if terminal or truncated:
            break
    total_rewards.append(rewards)


loading map from  ../templates/maps/game_map.yml
map size: [6, 6]
==== Player Character ====
name:  gomerin
level:  1
character class:  {'fighter': 1}
hp:  24
max hp:  24
ac:  18
speed:  30



==== Player Character ====
name:  rumblebelly
level:  1
character class:  {'fighter': 1}
hp:  24
max hp:  24
ac:  18
speed:  30



gomerin -> initiative roll: (9) + 5 value: 14.2
rumblebelly -> initiative roll: (8) + 5 value: 13.2
gomerin starts their turn.
gomerin moved to [5, 1] 5 feet
gomerin moved to [4, 1] 5 feet
gomerin dashes
gomerin moved to [3, 1] 5 feet
gomerin uses Second Wind with (2) + 1 healing
gomerin moved to [4, 1] 5 feet
gomerin moved to [3, 0] 5 feet
gomerin moved to [2, 0] 5 feet
gomerin moved to [1, 0] 5 feet
gomerin moved to [0, 1] 5 feet
gomerin moved to [0, 2] 5 feet
gomerin moved to [1, 3] 5 feet
gomerin moved to [0, 2] 5 feet
==== end turn ===
rumblebelly starts their turn.
==== current turn rumblebelly 24/24===
rumblebelly: rumblebelly attacks gomerin with longbow
rumbl

In [5]:

avg_rewards = sum(total_rewards) / len(total_rewards)
print(f"Average reward for Random Policy vs Heuristic: {avg_rewards}")

Average reward for Random Policy vs Heuristic: -24.0


In [6]:
prompt = GPT4Interfacer(debug=False, api_key="OPENAI api key here")
# prompt = LLama3Interface(url="http://202.92.159.241:8000/generate", debug=False)

total_rewards = []
for trials in tqdm.tqdm(range(NUM_TRAJECTORIES_TO_SAMPLE)):
    observation, info = env.reset(seed=42)
    terminal = False
    step = 0
    rewards = 0
    
    while not terminal and step < MAX_STEPS:
        step += 1
        action = prompt.select_action_for_state(observation, info)
        observation, reward, terminal, truncated, info = env.step(action)
        rewards += reward
        if terminal or truncated:
            break
    total_rewards.append(rewards)

  0%|          | 0/30 [00:00<?, ?it/s]

loading map from  ../templates/maps/game_map.yml
map size: [6, 6]
==== Player Character ====
name:  gomerin
level:  1
character class:  {'fighter': 1}
hp:  24
max hp:  24
ac:  18
speed:  30



==== Player Character ====
name:  rumblebelly
level:  1
character class:  {'fighter': 1}
hp:  24
max hp:  24
ac:  18
speed:  30



gomerin -> initiative roll: (9) + 5 value: 14.2
rumblebelly -> initiative roll: (8) + 5 value: 13.2
gomerin starts their turn.
{'action': 4}
gomerin moved to [5, 1] 5 feet
{'action': 6}
gomerin moved to [5, 2] 5 feet
{'action': 1}
gomerin rolls (8) + 7=15 using Longbow and misses rumblebelly
{'action': 5}
gomerin moved to [5, 3] 5 feet
{'action': 2}
gomerin moved to [4, 3] 5 feet
{'action': 9}
gomerin uses Second Wind with (2) + 1 healing
{'action': 2}
gomerin moved to [3, 3] 5 feet
==== end turn ===
rumblebelly starts their turn.
==== current turn rumblebelly 24/24===
rumblebelly: rumblebelly attacks gomerin with longbow
rumblebelly attacks gomerin using Longbow for 

  3%|▎         | 1/30 [00:23<11:31, 23.86s/it]

{'action': 2}
gomerin moved to [3, 3] 5 feet
==== end turn ===
rumblebelly starts their turn.
==== current turn rumblebelly 14/24===
rumblebelly: rumblebelly attacks gomerin with rapier
rumblebelly attacks gomerin using Rapier for ((5 + 1) + 5) 11 damage!
gomerin takes 11 damage!
gomerin died. :(
tpk
Result: tpk
loading map from  ../templates/maps/game_map.yml
map size: [6, 6]
==== Player Character ====
name:  gomerin
level:  1
character class:  {'fighter': 1}
hp:  24
max hp:  24
ac:  18
speed:  30



==== Player Character ====
name:  rumblebelly
level:  1
character class:  {'fighter': 1}
hp:  24
max hp:  24
ac:  18
speed:  30



gomerin -> initiative roll: (9) + 5 value: 14.2
rumblebelly -> initiative roll: (8) + 5 value: 13.2
gomerin starts their turn.
{'action': 1}
gomerin rolls (8) + 7=15 using Longbow and misses rumblebelly
{'action': 4}
gomerin uses Second Wind with (2) + 1 healing
{'action': 3}
gomerin moved to [5, 1] 5 feet
{'action': 4}
gomerin moved to [5, 0] 5 feet
{'action'

  7%|▋         | 2/30 [00:46<10:53, 23.32s/it]

...

{'action': 2}
gomerin moved to [0, 3] 5 feet
==== end turn ===
rumblebelly starts their turn.
==== current turn rumblebelly 2/24===
rumblebelly: rumblebelly attacks gomerin with longbow
rumblebelly attacks gomerin using Longbow for ((2) + 5) 7 damage!
gomerin takes 7 damage!
gomerin died. :(
tpk
Result: tpk
loading map from  ../templates/maps/game_map.yml
map size: [6, 6]
==== Player Character ====
name:  gomerin
level:  1
character class:  {'fighter': 1}
hp:  24
max hp:  24
ac:  18
speed:  30



==== Player Character ====
name:  rumblebelly
level:  1
character class:  {'fighter': 1}
hp:  24
max hp:  24
ac:  18
speed:  30



gomerin -> initiative roll: (9) + 5 value: 14.2
rumblebelly -> initiative roll: (8) + 5 value: 13.2
gomerin starts their turn.
{'action': 4}
gomerin moved to [5, 1] 5 feet
{'action': 3}
gomerin moved to [4, 1] 5 feet
{'action': 9}
gomerin moved to [5, 2] 5 feet
{'action': 6}
gomerin moved to [5, 3] 5 feet
{'action': 1}
gomerin rolls (8) + 7=15 using Longbow and mis

 97%|█████████▋| 29/30 [17:30<00:36, 36.29s/it]

{'action': 1}
gomerin moved to [1, 1] 5 feet
==== end turn ===
rumblebelly starts their turn.
==== current turn rumblebelly 24/24===
rumblebelly: rumblebelly attacks gomerin with rapier
rumblebelly attacks gomerin using Rapier for ((2) + 5) 7 damage!
gomerin takes 7 damage!
gomerin died. :(
tpk
Result: tpk
loading map from  ../templates/maps/game_map.yml
map size: [6, 6]
==== Player Character ====
name:  gomerin
level:  1
character class:  {'fighter': 1}
hp:  24
max hp:  24
ac:  18
speed:  30



==== Player Character ====
name:  rumblebelly
level:  1
character class:  {'fighter': 1}
hp:  24
max hp:  24
ac:  18
speed:  30



gomerin -> initiative roll: (9) + 5 value: 14.2
rumblebelly -> initiative roll: (8) + 5 value: 13.2
gomerin starts their turn.
{'action': 4}
gomerin moved to [5, 1] 5 feet
{'action': 9}
gomerin dashes
{'action': 6}
gomerin uses Second Wind with (4) + 1 healing
{'action': 5}
gomerin moved to [5, 2] 5 feet
{'action': 2}
gomerin moved to [4, 2] 5 feet
{'action': 5}
gom

100%|██████████| 30/30 [18:25<00:00, 36.84s/it]

{'action': 1}
gomerin attacks rumblebelly using Rapier for ((5) + 5) 10 damage!
rumblebelly takes 10 damage!
rumblebelly died. :(


In [7]:
avg_rewards = sum(total_rewards) / len(total_rewards)
print(f"Average reward for GPT-4 Policy vs Heuristic: {avg_rewards}")

Average reward for GPT-4 Policy vs Heuristic: -15.5
